### Analyzing health data from Samsung's devices over a span of 6 years to study sleep patterns

In [25]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
import json
import matplotlib.pyplot as plt
import seaborn as sns
samsung_base_dir = os.path.join('..', 'input', 'samsung')

In [29]:
samsung_csv_paths = glob(os.path.join( '*.csv'))
print(len(samsung_csv_paths), 'csvs found')

44 csvs found


In [41]:
sleep=pd.read_csv("sleep.csv")
sleep.head(5)

,date,has_sleep_data,efficiency,sleep_score,sleep_duration,start_time,start_time_epoch,update_time,update_time_epoch,create_time,create_time_epoch,time_offset,end_time,end_time_epoch,total_sleep
0,8/4/2018,1,88.42315,NaN,NaN,8/4/2018 19:43,1533411780,8/5/2018 4:18,1533442684,8/5/2018 4:18,1533442684,UTC+0300,8/5/2018 4:03,1533441780,8:20:00
1,8/5/2018,1,90.50279,NaN,NaN,8/5/2018 20:31,1533501060,8/5/2018 23:43,1533512622,8/5/2018 23:43,1533512622,UTC+0300,8/5/2018 23:29,1533511740,2:58:00
2,8/6/2018,1,78.46154,NaN,NaN,8/6/2018 12:26,1533558360,8/6/2018 13:44,1533563071,8/6/2018 13:44,1533563071,UTC+0300,8/6/2018 13:30,1533562200,1:04:00
3,8/6/2018,1,93.52518,NaN,NaN,8/6/2018 21:31,1533591060,8/7/2018 2:22,1533608566,8/7/2018 2:22,1533608566,UTC+0300,8/7/2018 2:08,1533607680,4:37:00
4,8/7/2018,1,86.59004,NaN,NaN,8/7/2018 19:28,1533670080,8/8/2018 0:02,1533686555,8/8/2018 0:02,1533686555,UTC+0300,8/7/2018 23:48,1533685680,4:20:00


In [42]:
#Convert 'date' from string to Datetime format
sleep['date'] = pd.to_datetime(sleep['date'], format='%m/%d/%Y')

In [43]:
#Convert 'total_sleep' from string to time format
sleep['total_sleep'] = pd.to_timedelta(sleep['total_sleep'])

In [44]:
# Assuming 'total_sleep' column is in hours (float) format
sleep['hours'] = sleep['total_sleep'].astype(int)  # Extract hours
sleep['minutes'] = (sleep['total_sleep'] - sleep['hours']) * 60  # Extract minutes

# Ensure 'hours' and 'minutes' are in integer format
sleep['hours'] = sleep['hours'].astype(int)
sleep['minutes'] = sleep['minutes'].astype(int)


TypeError: cannot astype a datetimelike from [timedelta64[ns]] to [int32]

In [37]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Assuming 'date' column is in datetime format
sleep['Month'] = sleep['date'].dt.to_period('M')

# Calculate total sleep time in minutes
sleep['total_sleep_minutes'] = sleep['hours'] * 60 + sleep['minutes']

# Calculate average sleep per month
average_sleep_per_month = sleep.groupby('Month')['total_sleep_minutes'].mean().reset_index()

# Grouping by month across all years
monthly_average_sleep = average_sleep_per_month.groupby(average_sleep_per_month['Month'].dt.month)['total_sleep_minutes'].mean().reset_index()

# Define a custom colormap ranging from light grey to black based on average sleep duration
min_avg = monthly_average_sleep['total_sleep_minutes'].min()
max_avg = monthly_average_sleep['total_sleep_minutes'].max()

cmap = mcolors.LinearSegmentedColormap.from_list("CustomMap", [(0, 'lightgrey'), (1, 'black')])

# Normalize average sleep durations to range between 0 and 1 for colormap usage
normalized = (monthly_average_sleep['total_sleep_minutes'] - min_avg) / (max_avg - min_avg)

# Plotting with Seaborn using the custom colormap
plt.figure(figsize=(40, 25))  # Maintain the larger figure size
ax = sns.barplot(x='Month', y='total_sleep_minutes', data=monthly_average_sleep, color='green')

# Rotate x-labels for better readability
plt.xticks(rotation=45)

# Adjust font sizes for better readability
plt.title('Average Sleep per Month', fontsize=40)
plt.xlabel('Month', fontsize=30)
plt.ylabel('Average Sleep (minutes)', fontsize=30)

# Change month numbers to month names in x-axis labels
ax.set_xticklabels([calendar.month_abbr[m] for m in range(1, 13)], fontsize=20, fontweight='bold')

# Set y-axis ticks starting from 30 minutes with bold font, skipping 0 hour
ticks = range(30, int(monthly_average_sleep['total_sleep_minutes'].max()) + 30, 30)
tick_labels = [f'{i // 60} hour' if i % 60 == 0 else i for i in ticks]
plt.yticks(ticks, tick_labels, fontsize=20, fontweight='bold')

# Customize grid lines (dashed lines)
plt.grid(axis='y', linestyle='--', linewidth=1.5)

# Save the plot as an image
plt.savefig('average_sleep_per_month_large_green.png', dpi=300)  # Maintain the higher DPI

# Show the plot
plt.show()


KeyError: 'hours'

In [38]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'date' column is in datetime format
sleep['Day'] = sleep['date'].dt.to_period('D')  # Group by day

# Calculate total sleep time in minutes
sleep['total_sleep_minutes'] = sleep['hours'] * 60 + sleep['minutes']

# Calculate average sleep per day
average_sleep_per_day = sleep.groupby('Day')['total_sleep_minutes'].mean().reset_index()

# Convert 'Day' column to datetime
average_sleep_per_day['Day'] = average_sleep_per_day['Day'].dt.to_timestamp()

# Group by month and calculate average sleep per month
average_sleep_per_month = average_sleep_per_day.groupby(average_sleep_per_day['Day'].dt.to_period('M'))['total_sleep_minutes'].mean().reset_index()

# Convert total sleep time to hours for the y-axis
average_sleep_per_month['total_sleep_hours'] = average_sleep_per_month['total_sleep_minutes'] / 60

# Plotting with Seaborn
plt.figure(figsize=(40, 25))  # Increase figure size
ax = sns.barplot(x='Day', y='total_sleep_hours', data=average_sleep_per_month, color='green')

# Rotate x-labels for better readability
plt.xticks(rotation=45)

# Adjust font sizes for better readability
plt.title('Average Sleep Each Month', fontsize=40)
plt.xlabel('Month', fontsize=30)
plt.ylabel('Average Sleep (hours)', fontsize=30)

# Set y-axis ticks starting from 0.5 hours with bold font
ticks = [i / 2 for i in range(int(average_sleep_per_month['total_sleep_hours'].max()) * 2 + 1)]
tick_labels = [f'{i:.1f} hour' for i in ticks]
plt.yticks(ticks, tick_labels, fontsize=20, fontweight='bold')

# Customize grid lines (dashed lines)
plt.grid(axis='y', linestyle='--', linewidth=1.5)

# Format x-axis labels to mmm-yy
ax.set_xticklabels(average_sleep_per_month['Day'].dt.strftime('%b-%y'), fontsize=20)

# Save the plot as an image
#plt.savefig('average_sleep_each_month_large_green_agg.png', dpi=300)  # Increase DPI for a sharper image

# Show the plot
plt.show()


KeyError: 'hours'

In [24]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'date' column is in datetime format
sleep['Day'] = sleep['date'].dt.to_period('D')  # Group by day

# Calculate total sleep time in minutes
sleep['total_sleep_minutes'] = sleep['hours'] * 60 + sleep['minutes']

# Calculate average sleep per day
average_sleep_per_day = sleep.groupby('Day')['total_sleep_minutes'].mean().reset_index()

# Convert 'Day' column to datetime
average_sleep_per_day['Day'] = average_sleep_per_day['Day'].dt.to_timestamp()

# Group by month and calculate average sleep per month
average_sleep_per_month = average_sleep_per_day.groupby(average_sleep_per_day['Day'].dt.to_period('M'))['total_sleep_minutes'].mean().reset_index()

# Convert total sleep time to hours for the y-axis
average_sleep_per_month['total_sleep_hours'] = average_sleep_per_month['total_sleep_minutes'] / 60

# Calculate overall mean sleep
overall_mean_sleep = average_sleep_per_month['total_sleep_hours'].mean()

# Plotting with Seaborn
plt.figure(figsize=(40, 25))  # Increase figure size
ax = sns.barplot(x='Day', y='total_sleep_hours', data=average_sleep_per_month, color='green')

# Rotate x-labels for better readability
plt.xticks(rotation=45)

# Adjust font sizes for better readability
plt.title('Average Sleep Each Month', fontsize=40)
plt.xlabel('Month', fontsize=30)
plt.ylabel('Average Sleep (hours)', fontsize=30)

# Set y-axis ticks starting from 0.5 hours with bold font
ticks = [i / 2 for i in range(int(average_sleep_per_month['total_sleep_hours'].max()) * 2 + 1)]
tick_labels = [f'{i:.1f} hour' for i in ticks]
plt.yticks(ticks, tick_labels, fontsize=20, fontweight='bold')

# Customize grid lines (dashed lines)
plt.grid(axis='y', linestyle='--', linewidth=1.5)

# Add a horizontal line for the overall mean sleep
plt.axhline(y=overall_mean_sleep, color='red', linestyle='--', linewidth=2, label='Overall Mean Sleep')

# Format x-axis labels to mmm-yy
ax.set_xticklabels(average_sleep_per_month['Day'].dt.strftime('%b-%y'), fontsize=20)

# Add a legend
plt.legend(fontsize=20)

# Save the plot as an image
#plt.savefig('average_sleep_each_month_large_green_agg.png', dpi=300)  # Increase DPI for a sharper image

# Show the plot
plt.show()


KeyError: 'date'